In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM,pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
# import transformers
# import sys

In [ ]:
llm_name = "chavinlo/alpaca-native"

tokenizer = LlamaTokenizer.from_pretrained(llm_name)
base_model = LlamaForCausalLM.from_pretrained(llm_name,
                                              offload_folder = "/Users/akshaymasurkar1/Desktop/llm_offload/",
                                              device_map="cpu")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=500,
    temperature=0.8,
    do_sample=True,
    top_p=0.95,
    repetition_penalty=1.2
)

template = """
Write a SQL Query given the table name {Table} and columns as a list {Columns} for the given question : 
{question}.
"""

prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])

local_llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt, llm=local_llm)


def get_llm_response(tble,question,cols):
    llm_chain = LLMChain(prompt=prompt, 
                         llm=local_llm
                         )
#     response= llm_chain.run({"Table" : tble,"question" :question, "Columns" : cols})
    response= llm_chain.invoke({"Table" : tble,"question" :question, "Columns" : cols})
    print(response)

In [ ]:
table='table_name'

columns=['col1','col2','col3']

question='write a query to select select customers who have xyz product'

get_llm_response(table,question,columns)